<a href="https://colab.research.google.com/github/ilkersigirci/ML-with-Colab/blob/master/final_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#DIFFERENT PREPROCESS
def prepare_doc(text): 
    #print(text) 
    text = text.lower() 
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, 
flags=re.MULTILINE) 
    #print(text) 
    text = re.sub(r'@\w+', ' ', text, flags=re.MULTILINE) 
    text = gensim.parsing.preprocessing.strip_tags(text) 
    text = gensim.parsing.preprocessing.strip_punctuation(text) 
    text = gensim.parsing.preprocessing.remove_stopwords(text) 
    text = gensim.parsing.preprocessing.strip_short(text) 
    text = gensim.parsing.preprocessing.strip_numeric(text) 
    text = gensim.parsing.preprocessing.stem_text(text) 
    #print(text) 
    return text

In [0]:
import os
import pprint
import tensorflow as tf
tf.reset_default_graph()
import pandas as pd
import numpy as np
import time

In [0]:
########################### ORIGINAL CSV #########################################

#pd.set_option('display.max_colwidth', -1) # for viewing questions with full length
dfPath = './support_forum_questions.csv'
df = pd.read_csv(dfPath, sep='|')
df.drop(["added", "login"], axis = 1,inplace=True)

df.dropna(axis = 0, how='any',inplace=True)
df.reset_index(drop=True,inplace=True)
df["id"] = df["id"].apply(str)

In [0]:
######################################### Glove ############################################## 
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

#!wget http://nlp.stanford.edu/data/glove.840B.300d.zip -P ./
#!unzip ./glove.840B.300d.zip -d ./
#!touch ./glove_word2vec.txt
#_ = glove2word2vec(r'./glove.840B.300d.txt', r"./glove_word2vec.txt")

glove_model = KeyedVectors.load_word2vec_format("./glove_word2vec.txt", binary=False)
#glove_model.save("./glove.model")

In [0]:
# just remove html tags

def removeHTML(text):
   
    text = re.sub('<[^<]+?>', ' ',                  text, flags=re.IGNORECASE)  # remove html
    text = re.sub('/(\r\n)+|\r+|\n+|\t+/i', '' ,    text, flags=re.IGNORECASE)  # remove \r \n
    text = re.sub("nbsp", "",                       text, flags=re.IGNORECASE)  # remove nbsp
   
    return text

df['original'] = df['question'] + ' === ' + df['details']
df['original'] = df['original'].apply(removeHTML)

In [0]:
#cleaning text
import re
import nltk
from string import punctuation
from nltk.stem import SnowballStemmer

# for lemmatizing
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# for stopwords
nltk.download('stopwords')

def clean(text):
    
    if pd.isnull(text):
        return ''
    
    # Empty question   
    if type(text) != str or text=='':
        return ''
    
    #Fixes that are specific to Jotform 
    text = re.sub('<[^<]+?>', ' ',                  text, flags=re.IGNORECASE)  # remove html
    text = re.sub('/(\r\n)+|\r+|\n+|\t+/i', '' ,    text, flags=re.IGNORECASE)  # remove \r \n
    text = re.sub("nbsp", "",                       text, flags=re.IGNORECASE)  # remove nbsp
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",     text, flags=re.IGNORECASE)  # remove tags
    text = re.sub("http\S*", "",                    text, flags=re.IGNORECASE)  # remove links
    text = re.sub("www\S*", "",                     text, flags=re.IGNORECASE)  # remove links
    text = re.sub("\S*@\S*\s?", "",                 text, flags=re.IGNORECASE)  # remove email
    text = re.sub("jo[a-z]*form", "jotform",        text, flags=re.IGNORECASE)  # fix1 jotform
    text = re.sub("jot[a-z]+", "jotform",           text, flags=re.IGNORECASE)  # fix2 jotform
    text = re.sub("jotform", "form",                text, flags=re.IGNORECASE)  # jotform to form
    text = re.sub("colour", "color",                text, flags=re.IGNORECASE)  # fix colour
    text = re.sub("authorizenet", "payment",        text, flags=re.IGNORECASE)
    text = re.sub("calender", "calendar",           text, flags=re.IGNORECASE)
    
    #Fix some words
    text = re.sub("autosuspended", "auto suspend",  text, flags=re.IGNORECASE)
    text = re.sub("autoresponses", "auto response", text, flags=re.IGNORECASE)
    text = re.sub("wordpresscom", "word press",     text, flags=re.IGNORECASE)
    text = re.sub("myforms", "my form",             text, flags=re.IGNORECASE)
    text = re.sub("onedrive", "cloud",              text, flags=re.IGNORECASE)
    text = re.sub("gmailcom", "email",              text, flags=re.IGNORECASE)
    text = re.sub("jo[a-z]*form", "form",           text, flags=re.IGNORECASE)
    text = re.sub("unrequire", "not require",       text, flags=re.IGNORECASE)
    text = re.sub("prepopulate[a-z]*", "populate",  text, flags=re.IGNORECASE)
    text = re.sub("sumbissions", "submission",      text, flags=re.IGNORECASE)
    text = re.sub("submissons", "submission",       text, flags=re.IGNORECASE)
    text = re.sub("mozilla\S*", "browser",          text, flags=re.IGNORECASE)
    text = re.sub("chrome\S*", "browser",           text, flags=re.IGNORECASE)
    text = re.sub("android","operating system",     text, flags=re.IGNORECASE)
    text = re.sub("ios", "operating system",        text, flags=re.IGNORECASE)
    
    #Fix Negative
    text = re.sub("n't", " not ",                   text, flags=re.IGNORECASE)
    text = re.sub("isnt", "is not",                 text, flags=re.IGNORECASE)
    text = re.sub("arent", "are not",               text, flags=re.IGNORECASE)
    text = re.sub("dont", "do not",                 text, flags=re.IGNORECASE)
    text = re.sub("doesnt", "does not",             text, flags=re.IGNORECASE)
    text = re.sub("didnt", "did not",               text, flags=re.IGNORECASE)
    text = re.sub("shouldnt", "should not",         text, flags=re.IGNORECASE)
    text = re.sub("hasnt", "has not",               text, flags=re.IGNORECASE)
    text = re.sub("havent", "have not",             text, flags=re.IGNORECASE)
    text = re.sub("wont", "will not",               text, flags=re.IGNORECASE)
    text = re.sub("cant", "can not",                text, flags=re.IGNORECASE)
    text = re.sub("couldnt", "could not",           text, flags=re.IGNORECASE)
    
    #Other fixes
    text = re.sub("\'s", " ",                       text, flags=re.IGNORECASE)
    text = re.sub("whats", "what is",               text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ",                 text, flags=re.IGNORECASE)    
    text = re.sub("i'm", "i am",                    text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ",                  text, flags=re.IGNORECASE)
    text = re.sub("\'d", " would ",                 text, flags=re.IGNORECASE)
    text = re.sub("\'ll", " will ",                 text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " example ",            text, flags=re.IGNORECASE)
    text = re.sub("e-mail", " email ",              text, flags=re.IGNORECASE)
    text = re.sub('\$', " dollar ",                 text, flags=re.IGNORECASE)
    text = re.sub('\%', " percent ",                text, flags=re.IGNORECASE)
    text = re.sub('\&', " and ",                    text, flags=re.IGNORECASE)
    
    text = ''.join([c for c in text if c not in punctuation]).lower()           # remove punc
    text = re.sub(" +"," ",                         text, flags=re.IGNORECASE)  # remove multiple space
    text = re.sub('[^\x00-\x7F]+', "",              text)                       # remove non-ascii characters    
    
    ######################### NON-ENGLISH REMOVAL ################################

    text = " ".join(word for word in nltk.wordpunct_tokenize(text) if word.lower() in glove_model.vocab and not word.isdigit())
    
    
    ############################### LEMMATIZE ####################################
    
    from nltk import word_tokenize, pos_tag
    from nltk.stem.wordnet import WordNetLemmatizer
    
    # n -> noun / v -> verb / a -> adjective / r -> adverb
    tokens = word_tokenize(text)
    convert_tag = lambda e: ('a' if e[0].lower() == 'j' else e[0].lower()) if e[0].lower() in ['n', 'r', 'v'] else 'n'    
    lemmatizer = WordNetLemmatizer()    
    text  = ' '.join([ lemmatizer.lemmatize(token, convert_tag(tag)) for token,tag in pos_tag(tokens) ])
    
    ############################### STOP WORDS ####################################
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    text =  " ".join(word for word in text.split() if word not in stop_words)

    return text

In [0]:
df['preprocessed_questions'] = df['question'].apply(clean)
df['preprocessed_details']   = df['details'].apply(clean)

In [0]:
#detect empty rows
df['question'] = df['question'].apply(lambda x: re.sub(" +"," ",x,flags=re.IGNORECASE))
df['details' ] = df['details' ].apply(lambda x: re.sub(" +"," ",x,flags=re.IGNORECASE))

# drop "" and " " rows
#df.drop(df[condition].index, axis=0, inplace=True)

df.drop(df[df['preprocessed_questions'] == "" ].index, axis=0, inplace=True)
df.drop(df[df['preprocessed_questions'] == " "].index, axis=0, inplace=True)
df.drop(df[df['preprocessed_details' ]  == "" ].index, axis=0, inplace=True)
df.drop(df[df['preprocessed_details' ]  == " "].index, axis=0, inplace=True)
df.reset_index(drop=True,inplace=True)

In [0]:
df['preprocessed_all'] = df['preprocessed_questions'] +' '+ df['preprocessed_details']

In [0]:
"""
df['less8'] = df['all'].apply(lambda x: " " if len(x.split()) <= 8 else "normal")
df.drop(df[df['less8'] == " "].index, axis=0, inplace=True)
df.reset_index(drop=True,inplace=True)
"""
# drop sentences less than 3 words
"""
short_indexes = []
for index,sentence in enumerate(df['all']):
    if(len(sentence.split()) <= 3):
        short_indexes.append(index)

df.drop(df.index[short_indexes],inplace = True)
df.reset_index(drop=True,inplace=True)
"""
df = df[df['preprocessed_all'].map( lambda x: len(x.split()) ) > 3]
df.reset_index(drop=True,inplace=True)

In [0]:
# drop non english rows  -> 6888
pip install langdetect
from langdetect import detect

non_english = []
for index,sentence in enumerate(df['preprocessed_all']):
    try:
        if detect(sentence) != "en":
            non_english.append(index)
    except:
        non_english.append(index)

df.drop(df.index[non_english],inplace = True)
df.reset_index(drop=True,inplace=True)

In [0]:
#remove rows that contains just stop words  -> 8
"""
df['preprocessed_all'] = df['preprocessed_all'].apply( lambda x: " ".join(word for word in x.split() if word not in stop_words) )
indexes = []
for index,sentence in enumerate(df['preprocessed_all']):
    if sentence == "": indexes.append(index)

indexes = np.array(indexes)
df.drop(df.index[indexes],inplace = True)
df.reset_index(drop=True,inplace=True)
"""

In [0]:
df.to_csv("./preprocessed.csv", sep='|')